In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
import pickle
import os

In [2]:
# 1. Load data with error handling
def load_data(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    return pd.read_csv(file_path)


data = load_data("C:/Users/syafi/Desktop/py-1/syafiq-project/dummy-data.csv")

In [3]:
# Display basic information about the dataset
print("Data Head:")
print(data.head())
print("\nData Description:")
print(data.describe())

Data Head:
   Heart Rate (bpm)  Breathing Rate (brpm)  Oxygen Saturation (%)  \
0              80.3                   12.2                   96.4   
1              73.1                   17.7                   95.9   
2              72.2                   18.0                   96.0   
3              70.6                   14.7                   95.1   
4              99.5                   19.5                   97.6   

   Blood Pressure (systolic)  Blood Pressure (diastolic)  Stress Index  \
0                      107.3                        74.2          39.6   
1                       92.4                        70.8          98.7   
2                      102.4                        75.6          45.3   
3                      110.0                        62.2          77.8   
4                      110.2                        73.0          57.3   

   Recovery Ability  PNS Index  SNS Index  RMSSD (ms)  SD2 (ms)  \
0                 0       -0.9        0.4        49.7      67.

In [4]:
# Disease classification based on thresholds
def classify_disease(row):
    """
    Classifies diseases for an individual based on health thresholds.

    :param row: A pandas Series representing a single individual's health data.
    :return: The disease associated with the individual.
    """
    if (
        row["Blood Pressure (systolic)"] >= 130
        or row["Blood Pressure (diastolic)"] >= 80
    ):
        return "Hypertension"

    if row["Blood Pressure (systolic)"] > 140 or row["Hemoglobin A1c (%)"] > 6.5:
        return "Atherosclerosis"

    if (
        row["Heart Rate (bpm)"] > 100
        or row["Heart Rate (bpm)"] < 60
        or row["Blood Pressure (systolic)"] > 140
        or row["Blood Pressure (diastolic)"] > 90
        or row["HRV SDNN (ms)"] < 50
    ):
        return "Cardiovascular Disease (CVD)"

    if row["Breathing Rate (brpm)"] > 20 or row["Oxygen Saturation (%)"] < 90:
        return "Respiratory Disease (COPD or Asthma)"

    if (
        row["HRV SDNN (ms)"] < 50
        or row["RMSSD (ms)"] < 30
        or row["Recovery Ability"] == 0
    ):
        return "Chronic Fatigue Syndrome (CFS)"

    if row["Hemoglobin A1c (%)"] > 6.5:
        return "Diabetes"

    if row["HRV SDNN (ms)"] > 100 or row["Mean RRi (ms)"] < 600:
        return "Arrhythmias"

    if row["Stress Index"] > 70 or row["SNS Index"] > 1.0:
        return "Stress-related Disorders"

    if row["PNS Index"] < -1.0 or row["SNS Index"] > 1.0:
        return "Autonomic Dysfunction"

    if (row["Gender (0-M;1-F)"] == 0 and row["Hemoglobin (g/dl)"] < 13.5) or (
        row["Gender (0-M;1-F)"] == 1 and row["Hemoglobin (g/dl)"] < 12.0
    ):
        return "Anaemia"

    return "Healthy"

 # Apply disease classification
data["Disease Classification"] = data.apply(classify_disease, axis=1)

In [5]:
# Display the class distribution
print("Class Distribution Before Sampling:")
print(data["Disease Classification"].value_counts())

Class Distribution Before Sampling:
Disease Classification
Hypertension                            285296
Cardiovascular Disease (CVD)             51327
Chronic Fatigue Syndrome (CFS)           51047
Atherosclerosis                          49081
Respiratory Disease (COPD or Asthma)     12458
Stress-related Disorders                   352
Arrhythmias                                282
Healthy                                     67
Autonomic Dysfunction                       65
Anaemia                                     25
Name: count, dtype: int64


In [6]:
# Encode labels using LabelEncoder
label_encoder = LabelEncoder()
data["Disease Classification"] = label_encoder.fit_transform(data["Disease Classification"])

In [7]:
# Save label mapping for reuse
label_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
with open("label_mapping.pkl", "wb") as f:
    pickle.dump(label_mapping, f)

In [8]:
# Split features and target
X = data.drop(columns=["Disease Classification"])
y = data["Disease Classification"]

# Display the first few rows of features and target
print("\nFeature Sample:")
print(X.head())
print("\nTarget Sample:")
print(y.head())


Feature Sample:
   Heart Rate (bpm)  Breathing Rate (brpm)  Oxygen Saturation (%)  \
0              80.3                   12.2                   96.4   
1              73.1                   17.7                   95.9   
2              72.2                   18.0                   96.0   
3              70.6                   14.7                   95.1   
4              99.5                   19.5                   97.6   

   Blood Pressure (systolic)  Blood Pressure (diastolic)  Stress Index  \
0                      107.3                        74.2          39.6   
1                       92.4                        70.8          98.7   
2                      102.4                        75.6          45.3   
3                      110.0                        62.2          77.8   
4                      110.2                        73.0          57.3   

   Recovery Ability  PNS Index  SNS Index  RMSSD (ms)  SD2 (ms)  \
0                 0       -0.9        0.4        49.7   

In [9]:
# Applying SMOTE for balancing the class distribution
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Display the class distribution after resampling
print("Class Distribution After SMOTE:")
print(pd.Series(y_resampled).value_counts())

Class Distribution After SMOTE:
Disease Classification
5    285296
7    285296
2    285296
4    285296
3    285296
8    285296
1    285296
0    285296
9    285296
6    285296
Name: count, dtype: int64


In [10]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, stratify=y_resampled, random_state=42
)

In [11]:
# Standardize features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Display scaled features
print("\nFirst 5 rows of Scaled Features:\n", X_train[:5])


First 5 rows of Scaled Features:
 [[ 0.87690214  1.5029835  -1.90574315  3.20507893  2.51798034  0.81129435
  -0.05463826 -1.60461037  1.50125974 -2.07631401 -0.64371508 -1.09981928
   1.27283733 -1.70995724  1.02091276  0.66616173 -0.60321693]
 [-1.32463128 -0.23392178  0.83711898  0.45092341  0.7674898  -0.79668184
  -1.51767221  0.89780967 -0.89952025  0.51741396  0.73535984 -0.51222028
  -0.52517188 -0.47279634 -0.2887977   0.91317706 -0.60321693]
 [-0.85945415 -0.10120899  0.50247428 -0.40680252 -0.59201968  0.95191793
  -0.05463826  0.11221336 -0.11094932 -0.09008737 -1.79301245 -0.1331218
  -0.10285491  0.48344541  0.29558427  0.77896775  1.6577784 ]
 [-0.84087114 -0.09382547  0.77281235  0.75448412 -1.44474411 -0.66760683
  -0.05463826  1.11885599  1.27264548  1.31334477  0.59727881 -1.3298029
  -0.00880713  1.4336082  -0.94478719  0.66616173 -0.60321693]
 [-0.79013608 -1.01377139  0.38787     0.29554921 -0.59189946  0.66953184
  -0.05463826 -0.16996165 -0.56330729  0.32240913

In [12]:
# Save the preprocessed data and scaler for later use
with open("preprocessed_data.pkl", "wb") as f:
    pickle.dump((X_train, X_test, y_train, y_test), f)

with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("\nPreprocessing completed and data saved.")


Preprocessing completed and data saved.
